In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import to_categorical
%matplotlib inline

Using TensorFlow backend.
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user10\Anaconda3\envs\P

In [2]:
df = pd.read_csv('./data/3406_2.csv')
col = ['DealPrice', 'Bid1', 'Ask1', 'Bid2', 'Ask2','Bid3', 'Ask3','Bid4', 'Ask4','Bid5', 'Ask5']
df_ = df[col].replace(to_replace=0, method='ffill')
df_return = (df_ / df_.shift(9) - 1)
col_ = []
for i in range(len(col)):
    col_.append(col[i] + '_return')
df_return.columns = col_
df = pd.concat([df, df_return], axis = 1)
df

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,Bid1_return,Ask1_return,Bid2_return,Ask2_return,Bid3_return,Ask3_return,Bid4_return,Ask4_return,Bid5_return,Ask5_return
0,311.0,1,1,309.5,311.0,2,16,309.0,311.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0,3,309.5,310.0,4,16,309.0,311.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0,5,310.0,310.5,10,1,309.5,311.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,310.5,2,2,310.0,310.5,6,1,309.5,311.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0,2,310.0,310.5,9,1,309.5,311.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513818,0.0,0,11,465.5,466.0,1,40,465.0,466.5,9,...,0.001075,-0.001072,0.001076,-0.001071,0.001078,-0.001070,0.001079,-0.001068,0.00108,-0.001067
513819,465.5,1,10,465.5,466.0,1,40,465.0,466.5,9,...,0.001075,-0.001072,0.001076,-0.001071,0.001078,-0.001070,0.001079,-0.001068,0.00108,-0.001067
513820,0.0,0,10,465.5,466.0,1,41,465.0,466.5,9,...,0.000000,-0.001072,0.000000,-0.001071,0.000000,-0.001070,0.000000,-0.001068,0.00000,-0.001067
513821,0.0,0,10,465.5,466.0,1,42,465.0,466.5,9,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [3]:
#data preprocessing
df['spread1'] = df['Ask1'] - df['Bid1']
df['spread2'] = df['Ask2'] - df['Bid2']
df['spread3'] = df['Ask3'] - df['Bid3']
df['spread4'] = df['Ask4'] - df['Bid4']
df['spread5'] = df['Ask5'] - df['Bid5']
df['mid2'] = (df['Ask2'] + df['Bid2']) / 2
df['mid3'] = (df['Ask3'] + df['Bid3']) / 2
df['mid4'] = (df['Ask4'] + df['Bid4']) / 2
df['mid5'] = (df['Ask5'] + df['Bid5']) / 2
df['a51_diff'] = df['Ask5'] - df['Ask1']
df['b15_diff'] = df['Bid1'] - df['Bid5']
df['a21_diff_abs'] = np.abs(df['Ask2'] - df['Ask1'])
df['a32_diff_abs'] = np.abs(df['Ask3'] - df['Ask2'])
df['a43_diff_abs'] = np.abs(df['Ask4'] - df['Ask3'])
df['a54_diff_abs'] = np.abs(df['Ask5'] - df['Ask4'])
df['b21_diff_abs'] = np.abs(df['Bid2'] - df['Bid1'])
df['b32_diff_abs'] = np.abs(df['Bid3'] - df['Bid2'])
df['b43_diff_abs'] = np.abs(df['Bid4'] - df['Bid3'])
df['b54_diff_abs'] = np.abs(df['Bid5'] - df['Bid4'])
df['a_mean'] = (df['Ask1'] +df['Ask2'] +df['Ask3'] +df['Ask4'] +df['Ask5']) / 5
df['b_mean'] = (df['Bid1'] +df['Bid2'] +df['Bid3'] +df['Bid4'] +df['Bid5']) / 5
df['aq_mean'] = (df['AskQty1'] +df['AskQty2'] +df['AskQty3'] +df['AskQty4'] +df['AskQty5']) / 5
df['bq_mean'] = (df['BidQty1'] +df['BidQty2'] +df['BidQty3'] +df['BidQty4'] +df['BidQty5']) / 5
df['p_acc_diff'] = (df['Ask1'] - df['Bid1']) +(df['Ask2'] - df['Bid2']) + (df['Ask3'] - df['Bid3']) + (df['Ask4'] - df['Bid4']) + (df['Ask5'] - df['Bid5'])
df['q_acc_diff'] = (df['AskQty1'] - df['BidQty1']) +(df['AskQty2'] - df['BidQty2']) + (df['AskQty3'] - df['BidQty3']) + (df['AskQty4'] - df['BidQty4']) + (df['AskQty5'] - df['BidQty5'])
#1 tick
df['da1_dt_1'] = df['Ask1'] - df['Ask1'].shift(1)
df['da2_dt_1'] = df['Ask2'] - df['Ask2'].shift(1)
df['da3_dt_1'] = df['Ask3'] - df['Ask3'].shift(1)
df['da4_dt_1'] = df['Ask4'] - df['Ask4'].shift(1)
df['da5_dt_1'] = df['Ask5'] - df['Ask5'].shift(1)
df['db1_dt_1'] = df['Bid1'] - df['Bid1'].shift(1)
df['db2_dt_1'] = df['Bid2'] - df['Bid2'].shift(1)
df['db3_dt_1'] = df['Bid3'] - df['Bid3'].shift(1)
df['db4_dt_1'] = df['Bid4'] - df['Bid4'].shift(1)
df['db5_dt_1'] = df['Bid5'] - df['Bid5'].shift(1)
#5 ticks
df['da1_dt_5'] = df['Ask1'] - df['Ask1'].shift(5)
df['da2_dt_5'] = df['Ask2'] - df['Ask2'].shift(5)
df['da3_dt_5'] = df['Ask3'] - df['Ask3'].shift(5)
df['da4_dt_5'] = df['Ask4'] - df['Ask4'].shift(5)
df['da5_dt_5'] = df['Ask5'] - df['Ask5'].shift(5)
df['db1_dt_5'] = df['Bid1'] - df['Bid1'].shift(5)
df['db2_dt_5'] = df['Bid2'] - df['Bid2'].shift(5)
df['db3_dt_5'] = df['Bid3'] - df['Bid3'].shift(5)
df['db4_dt_5'] = df['Bid4'] - df['Bid4'].shift(5)
df['db5_dt_5'] = df['Bid5'] - df['Bid5'].shift(5)
##10 ticks
df['da1_dt_10'] = df['Ask1'] - df['Ask1'].shift(10)
df['da2_dt_10'] = df['Ask2'] - df['Ask2'].shift(10)
df['da3_dt_10'] = df['Ask3'] - df['Ask3'].shift(10)
df['da4_dt_10'] = df['Ask4'] - df['Ask4'].shift(10)
df['da5_dt_10'] = df['Ask5'] - df['Ask5'].shift(10)
df['db1_dt_10'] = df['Bid1'] - df['Bid1'].shift(10)
df['db2_dt_10'] = df['Bid2'] - df['Bid2'].shift(10)
df['db3_dt_10'] = df['Bid3'] - df['Bid3'].shift(10)
df['db4_dt_10'] = df['Bid4'] - df['Bid4'].shift(10)
df['db5_dt_10'] = df['Bid5'] - df['Bid5'].shift(10)
#1 tick
df['daq1_dt_1'] = df['AskQty1'] - df['AskQty1'].shift(1)
df['daq2_dt_1'] = df['AskQty2'] - df['AskQty2'].shift(1)
df['daq3_dt_1'] = df['AskQty3'] - df['AskQty3'].shift(1)
df['daq4_dt_1'] = df['AskQty4'] - df['AskQty4'].shift(1)
df['daq5_dt_1'] = df['AskQty5'] - df['AskQty5'].shift(1)
df['dbq1_dt_1'] = df['BidQty1'] - df['BidQty1'].shift(1)
df['dbq2_dt_1'] = df['BidQty2'] - df['BidQty2'].shift(1)
df['dbq3_dt_1'] = df['BidQty3'] - df['BidQty3'].shift(1)
df['dbq4_dt_1'] = df['BidQty4'] - df['BidQty4'].shift(1)
df['dbq5_dt_1'] = df['BidQty5'] - df['BidQty5'].shift(1)
#5 ticks
df['daq1_dt_5'] = df['AskQty1'] - df['AskQty1'].shift(5)
df['daq2_dt_5'] = df['AskQty2'] - df['AskQty2'].shift(5)
df['daq3_dt_5'] = df['AskQty3'] - df['AskQty3'].shift(5)
df['daq4_dt_5'] = df['AskQty4'] - df['AskQty4'].shift(5)
df['daq5_dt_5'] = df['AskQty5'] - df['AskQty5'].shift(5)
df['dbq1_dt_5'] = df['BidQty1'] - df['BidQty1'].shift(5)
df['dbq2_dt_5'] = df['BidQty2'] - df['BidQty2'].shift(5)
df['dbq3_dt_5'] = df['BidQty3'] - df['BidQty3'].shift(5)
df['dbq4_dt_5'] = df['BidQty4'] - df['BidQty4'].shift(5)
df['dbq5_dt_5'] = df['BidQty5'] - df['BidQty5'].shift(5)
#10 ticks
df['daq1_dt_10'] = df['AskQty1'] - df['AskQty1'].shift(10)
df['daq2_dt_10'] = df['AskQty2'] - df['AskQty2'].shift(10)
df['daq3_dt_10'] = df['AskQty3'] - df['AskQty3'].shift(10)
df['daq4_dt_10'] = df['AskQty4'] - df['AskQty4'].shift(10)
df['daq5_dt_10'] = df['AskQty5'] - df['AskQty5'].shift(10)
df['dbq1_dt_10'] = df['BidQty1'] - df['BidQty1'].shift(10)
df['dbq2_dt_10'] = df['BidQty2'] - df['BidQty2'].shift(10)
df['dbq3_dt_10'] = df['BidQty3'] - df['BidQty3'].shift(10)
df['dbq4_dt_10'] = df['BidQty4'] - df['BidQty4'].shift(10)
df['dbq5_dt_10'] = df['BidQty5'] - df['BidQty5'].shift(10)
#response 放在最後一行
df['mid1'] = (df['Ask1'] + df['Bid1']) / 2
#drop nan
df = df.dropna()
df = df.reset_index(drop = True)
#b43_diff_abs and b54_diff_abs are all same values -> drop
#df = df.drop(['b43_diff_abs'], axis = 1)
#df = df.drop(['b54_diff_abs'], axis = 1)
from datetime import datetime
df['TxTime'] = pd.to_datetime(df['TxTime'])
df['TxTime'] = [str(x)[0:10] for x in df['TxTime']]
#some index's ask1 = 0, we change that mid1 to bid1
idx = df[df['Ask1'] == 0].index
for i in idx:
    df['mid1'].iloc[i] = df['Bid1'].iloc[i]
#3/23 3/24 data有缺值 -> drop
df = df[df['TxTime'] != '2020-03-23']
df = df[df['TxTime'] != '2020-03-24']
df = df.reset_index(drop = True)

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df['return_10t'] = df['mid1'].shift(-10) / df['mid1'] -1
df['return_10t'] = df['mid1'].shift(-10) / df['mid1'] -1
#1~10 predict 20/10的 return -> 11買賣 -> shift(1)
df['return_10t'] = df['return_10t'].shift(1)
df

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,daq3_dt_10,daq4_dt_10,daq5_dt_10,dbq1_dt_10,dbq2_dt_10,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,return_10t
0,398.0,755,2,397.0,398.0,17,1,396.5,398.5,9,...,144.0,3.0,60.0,-578.0,-804.0,-2.0,36.0,6.0,397.50,NaN
1,397.0,2,1,396.5,398.0,17,6,396.0,398.5,9,...,144.0,3.0,60.0,-580.0,-799.0,32.0,5.0,-2.0,397.25,-0.001887
2,0.0,0,2,396.5,398.0,17,6,396.0,398.5,9,...,144.0,3.0,60.0,-579.0,-800.0,32.0,5.0,-2.0,397.25,-0.001259
3,0.0,0,2,396.5,397.0,1,6,396.0,398.0,17,...,9.0,144.0,3.0,-578.0,-800.0,32.0,5.0,-2.0,396.75,-0.001888
4,0.0,0,2,396.5,397.0,2,6,396.0,398.0,17,...,9.0,144.0,3.0,-578.0,-796.0,32.0,5.0,-2.0,396.75,-0.000630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493215,0.0,0,11,465.5,466.0,1,40,465.0,466.5,9,...,0.0,1.0,0.0,-28.0,-9.0,-1.0,27.0,-65.0,465.75,NaN
493216,465.5,1,10,465.5,466.0,1,40,465.0,466.5,9,...,-17.0,27.0,6.0,-29.0,-9.0,-1.0,27.0,-65.0,465.75,NaN
493217,0.0,0,10,465.5,466.0,1,41,465.0,466.5,9,...,-17.0,27.0,6.0,-30.0,-8.0,-1.0,27.0,-65.0,465.75,NaN
493218,0.0,0,10,465.5,466.0,1,42,465.0,466.5,9,...,-17.0,27.0,6.0,0.0,2.0,-4.0,0.0,0.0,465.75,NaN


In [5]:
#後9筆nan drop掉
df = df.iloc[0:493211]
#drop TxTime
df2 = df.drop(['TxTime'], axis = 1)

In [6]:
#Def
#normalize
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm

#scale back to the true range
def true_range(df, x, response):
    X = (x * (df[response].max() - df[response].min())) + df[response].min()
    return X

#rolling windows
def train_windows(df, response, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        #扣掉response
        #記得最後一行要放response
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day][response]))
    return np.array(X_train), np.array(Y_train)

#Moving Average  
def MA(df, n):  
    MA = df.rolling(n).mean()
    return MA

#LSTM (when add the drop out layer in order to deal with overfitting)
def lstm_stock_model(shape):
    model = Sequential()
    #return sequence = True -> means the network will have long memory
    #first layer
    model.add(LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    #drop out layer(drop 50% of the previous one to avoid over-fitting)
    model.add(Dropout(0.5))
    #second layer
    model.add(LSTM(256, return_sequences=True))
    #add dense in every time step, dimension -> 1
    model.add(TimeDistributed(Dense(1)))
    #flatten: 多維 -> 一維
    model.add(Flatten())
    #dense layer
    model.add(Dense(10,activation='linear'))
    #1 -> output dimension
    model.add(Dense(1,activation='linear'))
    #using MSE
    model.compile(loss="mean_squared_error", optimizer="adam",metrics=['mean_squared_error'])
    model.summary()
    return model

#record transaction
def record_transaction(para, date, tick, action_side, price, equity_value, transaction_history):
    info = {
                'Date': date,
                'tick': tick,
                'Action': action_side,
                'Price': price,
                'Equity_value': equity_value
            }
    if para in transaction_history:
        transaction_history[para].append(info)
    else:
        transaction_history.update({para: [info]})    
    
    return transaction_history

#performance
def performance(equity_value_, transaction):
    #return
    Return = equity_value_.iloc[-1][0]
    print('return:', Return)
    #vol
    Std = equity_value_.std()[0]
    print('std:', Std)
    #sharp ratio
    Sharp_ratio = (Return - 0.0052) / Std
    print('sharp ratio:', Sharp_ratio)
    #nums of trading
    Trades = len(transaction) / 2
    print('trades:', Trades)
    #odds ratio
    R = 0
    for i in range(len(transaction)):
        if transaction[i]['Equity_value'] > 0:
            R += 1
    Odd = R / (len(transaction) / 2)
    print('odds ratio:', Odd)
    #mdd
    Mdd = (equity_value_.cummax() - equity_value_).max()[0]
    print('mdd:', Mdd)
    #skew
    Skew = equity_value_.skew()[0]
    print('skewness:', Skew)
     
    return Return, Std, Sharp_ratio, Trades, Odd, Mdd, Skew

#model performance
def model_performance(y, f):
    tot = np.sum(np.square(y - np.mean(y)))
    res = np.sum(np.square(y - f))
    r_squared = 1 - res / tot
    mse = np.mean(np.square(y - f)) * (1 / len(y))
    print('MSE:', mse)
    print('r-squared:', r_squared)    

In [7]:
#adjust the parameters in lstm
batch_size = [128, 256, 512]
validation_split = [0.1, 0.2, 0.3]
#normalize
norm_df = normalize(df2)
test = norm_df.iloc[372027: ]
test = test.reset_index(drop = True)
train = norm_df.iloc[0:372027]
X_train, Y_train = train_windows(train, 'return_10t', 10, 1)
X_test, Y_test = train_windows(test, 'return_10t', 10, 1)

In [8]:
#adding drop out layer(pct = 0.5)
#batch size = 256
#validation split = 0.3 (not letting val loss < loss)
#early stop -> val loss patience = 5
model_1 = lstm_stock_model(X_train.shape)
callback = EarlyStopping(monitor="val_mean_squared_error", patience=5, verbose=1, mode="min")
history_1 = model_1.fit(X_train, Y_train, epochs=500, batch_size=batch_size[1], validation_split=validation_split[2], callbacks=[callback], shuffle=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           385024    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             257       
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [9]:
X_t1 = model_1.predict(X_train)
Xt1 = model_1.predict(X_test)

In [10]:
print('train set')
model_performance(true_range(df2, Y_train, 'return_10t'), true_range(df2, X_t1, 'return_10t'))

train set
MSE: 1.4765449216691546e-12
r-squared: 0.07319339623292509


In [11]:
print('test set')
model_performance(true_range(df2, Y_test, 'return_10t'), true_range(df2, Xt1, 'return_10t'))

test set
MSE: 3.9906236678235905e-12
r-squared: 0.06526254324291003


In [12]:
#adding drop out layer(pct = 0.5)
#batch size = 256
#validation split = 0.3 (not letting val loss < loss)
#early stop -> val loss patience = 5
model_2 = lstm_stock_model(X_train.shape)
callback = EarlyStopping(monitor="val_mean_squared_error", patience=5, verbose=1, mode="min")
history_2 = model_2.fit(X_train, Y_train, epochs=500, batch_size=batch_size[0], validation_split=validation_split[2], callbacks=[callback], shuffle=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 256)           385024    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 1)             257       
_________________________________________________________________
flatten_2 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [13]:
X_t2 = model_2.predict(X_train)
Xt2 = model_2.predict(X_test)

In [14]:
print('train set')
model_performance(true_range(df2, Y_train, 'return_10t'), true_range(df2, X_t2, 'return_10t'))

train set
MSE: 1.5758609783867568e-12
r-squared: 0.010854095968409583


In [15]:
print('test set')
model_performance(true_range(df2, Y_test, 'return_10t'), true_range(df2, Xt2, 'return_10t'))

test set
MSE: 4.3399207514853974e-12
r-squared: -0.016554509632129344


In [ ]:
#adding drop out layer(pct = 0.5)
#batch size = 256
#validation split = 0.3 (not letting val loss < loss)
#early stop -> val loss patience = 5
model_3 = lstm_stock_model(X_train.shape)
callback = EarlyStopping(monitor="val_mean_squared_error", patience=10, verbose=1, mode="min")
history_3 = model_3.fit(X_train, Y_train, epochs=500, batch_size=batch_size[1], validation_split=validation_split[2], callbacks=[callback], shuffle=True)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 256)           385024    
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 1)             257       
_________________________________________________________________
flatten_4 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                